In [1]:
import sys
import pdb
from time import sleep
from logging import INFO

from vnpy.event import EventEngine
from vnpy.trader.setting import SETTINGS
from vnpy.trader.engine import MainEngine

from vnpy.gateway.huobif import HuobifGateway
from vnpy.app.cta_strategy import CtaStrategyApp
from vnpy.app.cta_strategy.base import EVENT_CTA_LOG
from vnpy.trader.object import SubscribeRequest
from vnpy.trader.constant import Exchange

SETTINGS["log.active"] = True
SETTINGS["log.level"] = INFO
SETTINGS["log.console"] = True

default_setting = {
    "API Key": "2be6e594-12edb3eb-rfhfg2mkl3-74a68",
    "Secret Key": "f3084214-4652d4fd-882f8eb2-b0256",
    "会话数": 3,
    "代理地址": "127.0.0.1",
    "代理端口": "1087",
}

ExchangeName = 'HUOBIF'

In [ ]:
def run():
    """
    Running in the child process.
    """
    SETTINGS["log.file"] = True

    event_engine = EventEngine()
    main_engine = MainEngine(event_engine)
    main_engine.add_gateway(HuobifGateway)
    cta_engine = main_engine.add_app(CtaStrategyApp)
    main_engine.write_log("主引擎创建成功")

    log_engine = main_engine.get_engine("log")
    event_engine.register(EVENT_CTA_LOG, log_engine.process_log_event)
    main_engine.write_log("注册日志事件监听")

    main_engine.connect(default_setting, ExchangeName)
    main_engine.write_log("连接接口")

    sleep(10)
    # main_engine.subscribe(sub, "HUOBIF")
    policy = 'test'
    cta_engine.init_engine()
    main_engine.write_log("策略初始化完成")
    # test_strategy
    cta_engine.init_all_strategies()
    sleep(10)   # Leave enough time to complete strategy initialization
    main_engine.write_log("策略全部初始化")

    pdb.set_trace()
    cta_engine.start_strategy(policy)
    main_engine.write_log("策略全部启动")

    while True:
        sleep(10)
        #print("进程存活")


if __name__ == "__main__":
    run()

2021-05-14 10:27:44,266  INFO: 主引擎创建成功
2021-05-14 10:27:44,272  INFO: 注册日志事件监听
2021-05-14 10:27:44,326  INFO: REST API启动成功
2021-05-14 10:27:44,354  INFO: 连接接口
2021-05-14 10:27:45,507  INFO: 交易Websocket API连接成功
2021-05-14 10:27:45,573  INFO: 合约信息查询成功
2021-05-14 10:27:45,654  INFO: 行情Websocket API连接成功
2021-05-14 10:27:45,815  INFO: 交易Websocket API登录成功
2021-05-14 10:27:46,114  INFO: ETH活动委托信息查询成功
2021-05-14 10:27:46,479  INFO: EOS活动委托信息查询成功
2021-05-14 10:27:46,816  INFO: ADA活动委托信息查询成功
2021-05-14 10:27:47,584  INFO: LINK活动委托信息查询成功
2021-05-14 10:27:48,407  INFO: BTC活动委托信息查询成功
2021-05-14 10:27:48,748  INFO: ETC活动委托信息查询成功
2021-05-14 10:27:49,267  INFO: FIL活动委托信息查询成功
2021-05-14 10:27:49,604  INFO: LTC活动委托信息查询成功
2021-05-14 10:27:49,940  INFO: BSV活动委托信息查询成功
2021-05-14 10:27:50,292  INFO: TRX活动委托信息查询成功
2021-05-14 10:27:51,174  INFO: BCH活动委托信息查询成功
2021-05-14 10:27:51,520  INFO: XRP活动委托信息查询成功
2021-05-14 10:27:52,295  INFO: DOT活动委托信息查询成功
2021-05-14 10:27:54,364  INFO: 创建策略失败，找不到策略类test
2021-05-14 10

> <ipython-input-2-4b6851f3d869>(31)run()
     29 
     30     pdb.set_trace()
---> 31     cta_engine.start_strategy(policy)
     32     main_engine.write_log("策略全部启动")
     33 

ipdb> s
--Call--
> /Users/shu/opt/anaconda3/lib/python3.8/site-packages/vnpy/app/cta_strategy/engine.py(702)start_strategy()
    700         self.write_log(f"{strategy_name}初始化完成")
    701 
--> 702     def start_strategy(self, strategy_name: str):
    703         """
    704         Start a strategy.

ipdb> n
> /Users/shu/opt/anaconda3/lib/python3.8/site-packages/vnpy/app/cta_strategy/engine.py(706)start_strategy()
    704         Start a strategy.
    705         """
--> 706         strategy = self.strategies[strategy_name]
    707         if not strategy.inited:
    708             self.write_log(f"策略{strategy.strategy_name}启动失败，请先初始化")

ipdb> p self.strategies
{}
